## Chat z pamięcią

### Instalacja bibliotek

In [8]:
!pip install -q langchain-openai python-dotenv langchain-core tiktoken

### Utworzenie modelu

In [9]:
from langchain_openai import ChatOpenAI
from dotenv import load_dotenv
load_dotenv()

llm = ChatOpenAI(
    model="gpt-4o-mini",  # możesz podmienić na inny wspierany model
    temperature=0         # 0 = maksymalna przewidywalność, dobre do testów pamięci
)

print("Model gotowy.")

Model gotowy.


### History
History – przechowuje i wstrzykuje do promptu całą historię rozmowy wiadomość po wiadomości.

In [10]:
from langchain.memory import ChatMessageHistory

# message history
history = ChatMessageHistory()

history.add_user_message("Buenos dias!")
history.add_ai_message("hello!")
history.add_user_message("Whats your name?")
history.add_ai_message("My name is GIGACHAT")
history.messages

[HumanMessage(content='Buenos dias!', additional_kwargs={}, response_metadata={}),
 AIMessage(content='hello!', additional_kwargs={}, response_metadata={}),
 HumanMessage(content='Whats your name?', additional_kwargs={}, response_metadata={}),
 AIMessage(content='My name is GIGACHAT', additional_kwargs={}, response_metadata={})]

In [12]:
from langchain_openai import ChatOpenAI
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain_core.runnables.history import RunnableWithMessageHistory

llm = ChatOpenAI(model="gpt-4o-mini", temperature=0)

prompt = ChatPromptTemplate.from_messages([
    ("system", "Prowadzisz przyjazną rozmowę i pamiętasz kontekst. "
               "Zawsze odpowiadaj po polsku."),
    MessagesPlaceholder("history"),          # <-- kluczowe!
    ("user", "{input}"),
])

chain = prompt | llm  # bez StrOutputParser, zapisze AIMessage do historii

store = {}
def get_history(session_id: str):
    if session_id not in store:
        store[session_id] = InMemoryChatMessageHistory()
    return store[session_id]

chain_with_memory = RunnableWithMessageHistory(
    chain,
    get_session_history=get_history,
    input_messages_key="input",
    history_messages_key="history",
    # output_messages_key domyślne działa; zostawiamy
)

sid = "demo-session-1"

resp1 = chain_with_memory.invoke(
    {"input": "Cześć! Mam na imię Michał."},
    config={"configurable": {"session_id": sid}}
)
print(resp1.content)

resp2 = chain_with_memory.invoke(
    {"input": "Jak mam na imię?"},
    config={"configurable": {"session_id": sid}}
)
print(resp2.content)


Cześć, Michał! Miło mi cię poznać. Jak mija twój dzień?
Masz na imię Michał. Jakie masz plany na dzisiaj?


### Memory
Memory – zarządza kontekstem rozmowy w bardziej zaawansowany sposób, np. buforuje tylko ostatnie N wiadomości albo wyodrębnia istotne fakty.

In [14]:
from langchain_openai import ChatOpenAI
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain_core.output_parsers import StrOutputParser
from langchain_core.chat_history import InMemoryChatMessageHistory
from langchain_core.runnables.history import RunnableWithMessageHistory

# --- Model ---
llm = ChatOpenAI(model="gpt-4o-mini", temperature=0)

# --- Prompt with history slot ---
prompt = ChatPromptTemplate.from_messages([
    ("system", "You are a helpful assistant. Respond in the same language as the user and greet him/her using his/her name."),
    MessagesPlaceholder("history"),
    ("human", "{input}")
])

chain = prompt | llm | StrOutputParser()

store = {}

def get_session_history(session_id: str) -> InMemoryChatMessageHistory:
    if session_id not in store:
        store[session_id] = InMemoryChatMessageHistory()
    return store[session_id]

conversation = RunnableWithMessageHistory(
    chain,
    get_session_history=get_session_history,
    input_messages_key="input",
    history_messages_key="history",
    output_messages_key="output"
)

# --- Usage ---
cfg = {"configurable": {"session_id": "test-session"}}
history = get_session_history("test-session")

# preload conversation history (like ConversationBufferMemory did)
history.add_user_message("Buenos dias!")
history.add_ai_message("Hello!")
history.add_user_message("My name is Michał. What's your name?")
history.add_ai_message("My name is GIGACHAT")

# now query
print(conversation.invoke({"input": "Buenos Dias!"}, cfg))


¡Buenos días, Michał! ¿Cómo estás hoy?


### Summary
Summary (memory) – zamiast pełnej historii przekazuje do modelu skondensowane podsumowanie wcześniejszych rozmów, co oszczędza tokeny i ułatwia skalowanie długich dialogów.

In [17]:
from langchain_openai import ChatOpenAI
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain_core.output_parsers import StrOutputParser
from langchain_core.chat_history import InMemoryChatMessageHistory
from langchain_core.runnables.history import RunnableWithMessageHistory

llm = ChatOpenAI(model="gpt-4o-mini", temperature=0)

# --- Summarizer chain ---
summarizer_prompt = ChatPromptTemplate.from_messages([
    ("system", "Summarize the following conversation briefly:"),
    ("human", "{conversation}")
])
summarizer = summarizer_prompt | llm | StrOutputParser()

# --- Conversation prompt (with summary injected) ---
conversation_prompt = ChatPromptTemplate.from_messages([
    ("system", "You are a helpful assistant. Here is the summary of prior conversation:\n{summary}"),
    MessagesPlaceholder("history"),
    ("human", "{input}")
])
conversation_chain = conversation_prompt | llm | StrOutputParser()

# --- Storage for sessions ---
store = {}

def get_history(session_id: str) -> InMemoryChatMessageHistory:
    if session_id not in store:
        store[session_id] = InMemoryChatMessageHistory()
    return store[session_id]

# --- Wrapper with history ---
chain_with_memory = RunnableWithMessageHistory(
    conversation_chain,
    get_session_history=get_history,
    input_messages_key="input",
    history_messages_key="history",
    output_messages_key="output"
)

# --- Function to get summary before invoking ---
def get_summary(session_id: str) -> str:
    history = get_history(session_id).messages
    if not history:
        return "No previous conversation."
    return summarizer.invoke({"conversation": history})

# --- Usage ---
sid = "demo-summary"

# First round
cfg = {"configurable": {"session_id": sid}}
summary = get_summary(sid)
print(chain_with_memory.invoke({"input": "Cześć! Mam na imię Michał.", "summary": summary}, cfg))

# Next round, now with updated summary
summary = get_summary(sid)
print(chain_with_memory.invoke({"input": "Jak mam na imię?", "summary": summary}, cfg))


Cześć, Michał! Jak mogę Ci dzisiaj pomóc?
Masz na imię Michał. Jakie masz pytania lub w czym mogę Ci pomóc?
